<a href="https://colab.research.google.com/github/KurniaKhaikal/Cifar10k_TransferLearning/blob/main/TransferLearning_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import copy
import numpy as np
import os
import json
import cv2
import sklearn.metrics as metric
from tensorflow.keras.models import Model
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"khaikal","key":"9e492a7553b8e713debda6f4f6810151"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d oxcdcd/cifar10

100% 175M/175M [00:01<00:00, 153MB/s]
100% 175M/175M [00:01<00:00, 161MB/s]


In [5]:
import os
import zipfile

local_zip = '/content/cifar10.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [7]:
!pip install split-folders

In [8]:
import splitfolders


splitfolders.ratio("/content/cifar10/test", output="/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2",
    seed=1337, ratio=(.9, .1), group_prefix=None, move=False)

Copying files: 10000 files [01:49, 91.09 files/s]


In [9]:
def euclidean(a, b):
	return np.linalg.norm(a - b)

In [10]:
def perform_search(query_features, indexed_train, max_results=5):
	retrieved = []
	for idx in range(0, len(indexed_train["features"])):
		distance = euclidean(query_features, indexed_train["features"][idx])
		retrieved.append((distance, idx))
	retrieved = sorted(retrieved)[:max_results]
	return retrieved

In [11]:
base_dataset = "/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2"
class_dir = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
IMAGE_SIZE = (224, 224)

In [12]:
# indexing file images
dataset_test = []
for class_item in class_dir:
    cur_dir = os.path.join(base_dataset, 'val', class_item)
    for file in os.listdir(cur_dir):
        dataset_test.append(os.path.join(cur_dir, file))

In [13]:
print("len to retrieving:", len(dataset_test))

len to retrieving: 1000


In [14]:
print("[INFO] load images Corel-Image 1k dataset...")
#  load images
val_images = []
for image_path in dataset_test:
    if ".jpg" or ".jpeg" in image_path:
        image = cv2.imread(image_path)
        image = cv2.resize(image, IMAGE_SIZE)
        val_images.append(image)

[INFO] load images Corel-Image 1k dataset...


In [18]:
print("[INFO] normalization...")
val_x = np.array(val_images).astype("float32") / 255.0

[INFO] normalization...


In [19]:
from tensorflow.keras.applications import VGG16
modelVGG16 = VGG16(weights="imagenet")
model = Model(inputs=modelVGG16.input, outputs=modelVGG16.get_layer("fc1").output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [20]:
features = model.predict(val_x)

In [21]:
with open('/content/drive/MyDrive/Proposal2/Cifar10/Cifar_dataset/cifar10/Base2/feature_extraction.json') as f:
  training_indexed = json.load(f)

In [41]:
query_indexes = list(range(0, val_x.shape[0]))
label_builder = list(np.unique(training_indexed["labels"]))
class_builder = {label_unique:[] for label_unique in label_builder}
recalls = copy.deepcopy(class_builder)
precisions = copy.deepcopy(class_builder)
# loop over the testing indexes
for i in query_indexes:
    queryFeatures = features[i]
    results = perform_search(queryFeatures, training_indexed, max_results=5)
    labels_ret = [training_indexed["labels"][r[1]] for r in results]
    label_true = dataset_test[i].split("/")[8]
    label_trues = [label_true for _ in labels_ret]
    recall = metric.recall_score(label_trues, labels_ret, average='weighted')
    precision = metric.precision_score(label_trues, labels_ret, average='weighted')
    recalls[label_true].append(recall)
    precisions[label_true].append(precision)

KeyboardInterrupt: ignored

In [ ]:
print("recall values:")
comb_recall, comb_precision = [], []
for key in recalls.keys():
    average_val = np.average(recalls[key])
    print(key, average_val)
    comb_recall.append(average_val)
print("combined recall", np.average(comb_recall))

print("\nprecision values:")
for key in precisions.keys():
    average_val = np.average(precisions[key])
    print(key, average_val)
    comb_precision.append(average_val)
print("combined precision", np.average(comb_precision))